# Equal Weight S&P 500
https://www.youtube.com/watch?v=xfzGZB4HhEE&ab_channel=freeCodeCamp.org

you actually never followed that yt video at all.

In [25]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime

# preamble

- [x] establish api key
- [x] establish variables
- [x] get symbols for all sp500 
- [x] run test query to confirm errorless execution

In [26]:
# establish api key
keys_file = open("alphaVantage.txt")
api_key = keys_file.readlines()[0]

In [27]:
#S&P 500 symbols
sp500_symbols = pd.read_csv('https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv')['Symbol']

In [28]:
# # # test query. if things break, see if this is still working
# ticker = 'ABMD'
# interval = '5min'
# function = 'TIME_SERIES_INTRADAY'
# # run test query
# ibm = requests.get(f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}').json()
# # aapl

# build dataframe of 5 minute moves for yesterday

In [29]:
bandit = list(sp500_symbols)#[:6]
#bandit

In [30]:
def get_5min(ticker, interval, function):
    thing = {}
    
    for i in ticker:
        try:
            ticker = str(i)
            function = str(function)
            api_info = requests.get(f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}').json()
            shortcode = api_info['Meta Data']['2. Symbol'] # get the symbol e.g. "AAPL"
            five_min = api_info['Time Series (5min)'] # get the 5 minute intervals THIS WILL BREAK IF YOU CHANGE THE FUNCTION
            n = api_info['Time Series (5min)'] # get the 5 minute intervals

            close_price = {} #build dictionary of times and prices
            for i in five_min:
                close_price[i.split(' ')[1]] = five_min[i]['4. close']

            out = {shortcode: close_price}
            thing.update(out)
            time.sleep(13)
    
        except:
            continue

        df = pd.DataFrame.from_dict(thing,orient='index').T
        df.sort_index(inplace=True)

    return df

In [31]:
df_idk = get_5min(bandit, '5min', 'TIME_SERIES_INTRADAY')
df_idk

,MMM,GOOGL,GOOG,AMZN,AAL,AAPL,T,BKR,CCL,CVX,...,TRMB,TYL,UDR,URI,VFC,WAT,WST,WDC,WRK,WHR
04:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19:40:00,NaN,108.0000,108.4600,122.2000,13.6800,153.9500,18.3700,NaN,9.2300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19:45:00,NaN,108.0000,108.4500,122.1600,13.6800,153.9100,18.3900,NaN,9.2500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19:50:00,133.8900,107.9700,108.5000,122.2000,13.6600,153.8900,NaN,NaN,9.2500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19:55:00,NaN,108.0000,108.4800,122.1500,13.6800,153.8900,18.3800,24.0100,9.2500,NaN,...,NaN,NaN,NaN,NaN,47.5000,NaN,NaN,NaN,NaN,NaN


In [32]:
# backfill, forwardfill, save to csv
today = datetime.today().strftime('%Y-%m-%d')
df_idk = df_idk.ffill().bfill()
df_idk.to_csv(f'./data/raw/sp500-{today}.csv')